## 약사 챗봇 만들기

In [ ]:
# 데이터는 https://www.kaggle.com/datasets/ujjwalaggarwal402/medicine-dataset 에서 가져옴

In [ ]:
# 데이터는 https://nedrug.mfds.go.kr/pbp/CCBGA01/getItem?infoType=C&totalPages=8&limit=10&searchYn=true&page=1&&openDataInfoSeq=11 에서 가져옴

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

#cd command needs '%' not '!', while other Unix commands need '!'
%cd ./drive/My\ Drive/2024_2배울랑교/Medicine_chatbot
!pwd
!ls -l

Mounted at /content/drive
/content/drive/My Drive/2024_2배울랑교/Medicine_chatbot
/content/drive/My Drive/2024_2배울랑교/Medicine_chatbot
total 74
drwx------ 2 root root  4096 Nov 21 05:56  data
drwx------ 2 root root  4096 Nov 23 00:17  medicine_faiss_index_folder
-rw------- 1 root root 67061 Nov 26 11:15 '약사 챗봇 만들기.ipynb'


In [ ]:
!pip install langchain openai chromadb tiktoken pypdf unstructured sentence-transformers langchain-community
!pip install langchain langchain-community langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 32.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd

## 데이터 전처리

In [ ]:
from langchain.document_loaders import TextLoader

In [ ]:
medicine_file_path = './data/Medicine_data.txt'
loader = TextLoader(medicine_file_path, encoding='cp949')

docs = loader.load()
# page_content에서 탭을 공백으로 바꿉니다.
for doc in docs:
    doc.page_content = doc.page_content.replace('\t', ' ')

# 첫 번째 문서 데이터 접근
print(docs[0].page_content[0:1000])

품목일련번호 품목명 업소일련번호 업소명 성상 큰제품이미지 표시앞 표시뒤 의약품제형 색상앞 색상뒤 분할선앞 분할선뒤 크기장축 크기단축 크기두께 이미지생성일자(약학정보원) 분류번호 분류명 전문일반구분 품목허가일자 제형코드명 표기내용앞 표기내용뒤 표기이미지앞 표기이미지뒤 표기코드앞 표기코드뒤 변경일자 사업자번호
200808876 가스디알정50밀리그램(디메크로틴산마그네슘) 19540006 일동제약(주) 녹색의 원형 필름코팅정 https://nedrug.mfds.go.kr/pbp/cmn/itemImageDownload/147426403087300104 IDG  원형 연두    7.6 7.6 3.6 20100326 02390 기타의 소화기관용약 전문의약품 20080820 당의정        6828500435
200808877 페라트라정2.5밀리그램(레트로졸) 19560004 (주)유한양행 어두운 황색의 원형 필름코팅정 https://nedrug.mfds.go.kr/pbp/cmn/itemImageDownload/147426403087300107 YH LT 원형 노랑    6.1 6.1 3.5 20100429 04210 항악성종양제 전문의약품 20080820 필름코팅정        1188100601
200808948 졸뎀속붕정(졸피뎀타르타르산염) 20080422 보령제약(주) 흰색의 원형 구강붕해정제 https://nedrug.mfds.go.kr/pbp/cmn/itemImageDownload/147426403087300128   원형 하양    9 9 4 20100701 01120 최면진정제 전문의약품 20080825 정제        1348502031
200809076 가스프렌정(모사프리드시트르산염이수화물) 19760002 경동제약(주) 분할선을 가진 흰색의 장방형 필름코팅정제 https://nedrug.mfds.go.kr/pbp/cmn/itemImageDownload/147426403087300143 KD 분할선 장방형 하양   - 9.1 4.5 3.4 201009

In [ ]:
lines = docs[0].page_content.split('\n')
print(lines[1])

200808876 가스디알정50밀리그램(디메크로틴산마그네슘) 19540006 일동제약(주) 녹색의 원형 필름코팅정 https://nedrug.mfds.go.kr/pbp/cmn/itemImageDownload/147426403087300104 IDG  원형 연두    7.6 7.6 3.6 20100326 02390 기타의 소화기관용약 전문의약품 20080820 당의정        6828500435


## 모델 만들기

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
os.environ["OPENAI_API_KEY"] = "Your_OpenAi_Key"

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini"
    )

In [ ]:
# 대화체 연쇄 만들기
conversation_chain = ConversationChain(llm=llm)

<ipython-input-8-4825e14ff2f7>:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation_chain = ConversationChain(llm=llm)
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:212: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 51.3 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from langchain.docstore.document import Document

documents = [Document(page_content=line) for line in lines]

# OpenAI 임베딩을 사용하여 데이터셋을 벡터화
embeddings = OpenAIEmbeddings()

# FAISS 인덱스에 데이터 삽입
faiss_index = FAISS.from_documents(documents, embeddings)

# 검색 기반 QA 시스템 설정
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=faiss_index.as_retriever())

<ipython-input-12-c6c2e6426dc7>:9: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


## 모델 저장

In [ ]:
# FAISS 인덱스 저장
faiss_index.save_local("./data/medicine_faiss_index_folder")

NameError: name 'faiss_index' is not defined

##모델 불러오기


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 73.8 MB/s eta 0:00:00


In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
os.environ["OPENAI_API_KEY"] = "Your_OpenAi_Key"

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini"
    )
# 여기에도 뭔가 조절하는게 많음 temperature가 0에 가까우면 확실한 답만, 2에 가까울수록 창의적인 답을 많이 도출해내는 함수도 잇음.

In [ ]:
# 대화체 연쇄 만들기
conversation_chain = ConversationChain(llm=llm)

<ipython-input-10-4825e14ff2f7>:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation_chain = ConversationChain(llm=llm)
/usr/local/lib/python3.10/dist-packages/pydantic/main.py:212: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [ ]:
# 저장된 인덱스 불러오는 법
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from langchain.docstore.document import Document

embeddings = OpenAIEmbeddings()  # 임베딩 모델 초기화

faiss_index = FAISS.load_local("./data/medicine_faiss_index_folder", embeddings, allow_dangerous_deserialization=True) # allow_dangerous_deserialization를 True로 안하면 보안조치로 신뢰할 수 없는 파일을 함부로 사용할 수 없도록 일부러 ValueError를 발생시킴

# 검색 기반 QA 시스템 설정
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=faiss_index.as_retriever(search_kwargs={"k" : 5}))

<ipython-input-11-7101a34a0f65>:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()  # 임베딩 모델 초기화


## 질문

In [ ]:
# 사용자 질문 처리
user_question = "분홍, 원형, SCD. 이게 무슨 약이야?"

# QA 체인으로 질문 처리
response = qa_chain.run(user_question)

print(response)

분홍색의 원형 필름코팅정제인 "에스부펜정(덱시부프로펜)"입니다. 이 약은 해열, 진통, 소염제에 해당하며 일반의약품입니다.


In [ ]:
# 사용자 질문 처리
user_question = "에스부펜정(덱시부프로펜)에 대해 설명하고 사진을 볼 수 있는 링크를 달아줘."

# QA 체인으로 질문 처리
response = qa_chain.run(user_question)

print(response)

에스부펜정(덱시부프로펜)은 삼천당제약(주)에서 제조한 해열, 진통, 소염제입니다. 이 약물은 분홍색의 원형 필름코팅 정제로, 일반의약품으로 분류됩니다.

사진을 볼 수 있는 링크는 다음과 같습니다: [사진 보기](https://nedrug.mfds.go.kr/pbp/cmn/itemImageDownload/147426768452000008)
